In [1]:
# import os, sys
# import yaml 
# import json 
# import glob
# import subprocess
# from shutil import copyfile
# from datetime import datetime

# import pandas as pd
# import shap

# from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report
# ,roc_auc_score, log_loss, precision_recall_curve, auc, precision_score, recall_score
# from sklearn.preprocessing import Binarizer

In [3]:
def make_local_importance(input_df):
    df = input_df.copy()
    pred1_df = df[df['예측']==1]
    shap_cols = [col for col in pred1_df.columns if '(SHAP)' in col]
    columns = [col[6:] for col in shap_cols]
    total_dic = {}
    
    for i in range(len(pred1_df)):
        local_shap = pred1_df[shap_cols].iloc[i, :].values
        sorted_idx = local_shap.argsort()[::-1]
        key = pred1_df['key'].values[i]
        
        local_dic = {}
        for j in range(10):
            col = columns[sorted_idx[j]]
            importance = np.round(local_shap[sorted_idx[j]], 4)
            actual_local = pred1_df[col].values[i]
            local_dic[j+1] = f'{col}({actual_local}/{importance})'
        total_dic[key] = local_dic
        
    return total_dic


In [ ]:
def make_local_importance2(input_df):
    df = input_df.copy()
    shap_cols = [col for col in df.columns if '(SHAP)' in col]
    columns = [col[6:] for col in shap_cols]
    total_dic = {}
    
    for i in range(len(df)):
        local_shap = df[shap_cols].iloc[i, :].values
        sorted_idx = local_shap.argsort()[::-1]
        key = df['key'].values[i]
        
        local_dic = {}
        for j in range(10):
            col = columns[sorted_idx[j]]
            importance = np.round(local_shap[sorted_idx[j]], 4)
            actual_local = df[col].values[i]
            local_dic[j+1] = f'{col}({actual_local}/{importance})'
        total_dic[key] = local_dic
        
    df_shap = pd.DataFrame(total_dic)
    df_shap = df_shap.T.reset_index()
    df_shap.drop('index', axis=1, inplace=True)
    
    columns =[]
    for i in range(1, 11):
        columns.append(f'rank{i}')
        
    df_shap.columns = columns
    shap_cols = [col for col in df.columns if '(SHAP)' in col]
    
    df_add_shap = pd.concat([df.drop(shap_cols, axis=1), df_shap], axis=1)
    
    return df_add_shap